In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests

import dotenv

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.3
BeautifulSoup   4.13.3


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for everything goes to 1.', display_name='SkynetAndChill.com', followers_count=332, follows_count=347, indexed_at='2024-11-21T15:40:08.470Z', joined_via_starter_p

In [5]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [6]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [7]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [8]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [9]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [10]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [11]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [12]:
imgdir = 'download_images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('articles.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname: row.site_name for row in sites_df.itertuples()}
conn.close()


In [13]:
def parse_feed():
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.rstrip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                parsed_url = urlparse(post_url)
                hostname = parsed_url.hostname
                site_name = sites_dict.get(hostname, hostname)

            md_str = f"[{post_str}]({post_url})  - {site_name}"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed()
len(display_dict)


https://indicator.media/p/the-top-community-notes-contributor-is-an-antivirus-startup-9354
https://www.bloomberg.com/news/newsletters/2025-05-25/how-jony-ive-openai-deal-will-impact-apple-new-details-on-apple-s-ios-redesign-mb3lwu45?cmpid=BBD052525_POWERON&utm_medium=email&utm_source=newsletter&utm_term=250525&utm_campaign=poweron
https://www.ft.com/content/e0bbbaf7-9f70-44cf-9bbf-077152d605a5
https://www.loudersound.com/news/ice-nine-kills-sarcastically-apologise-ai-image-2025
https://techxplore.com/news/2025-05-ai-sources-driver-stress-paving.html#google_vignette
https://timesofindia.indiatimes.com/technology/artificial-intelligence/tesla-ceo-elon-musks-one-word-reply-to-openais-ai-model-refusing-to-shutdown-on-command/articleshow/121414780.cms
https://www.statnews.com/2025/05/27/artificial-intelligence-bioterrorism-deepfake-public-health-threat/
https://www.futurehouse.org/research-announcements/demonstrating-end-to-end-scientific-discovery-with-robin-a-multi-agent-system
https://di

79

In [14]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))


0. [Using AI to community-note crypto scams on social media](https://indicator.media/p/the-top-community-notes-contributor-is-an-antivirus-startup-9354)  - indicator.media

---

1. [Apple said to plan smart home device later this year despite Apple Intelligence delays](https://www.bloomberg.com/news/newsletters/2025-05-25/how-jony-ive-openai-deal-will-impact-apple-new-details-on-apple-s-ios-redesign-mb3lwu45?cmpid=BBD052525_POWERON&utm_medium=email&utm_source=newsletter&utm_term=250525&utm_campaign=poweron)  - Bloomberg

---

2. 
is nothing sacred, not even shilling your metalcore merch and swag drop



---

3. [Creator of Succession has a new œuvre about tech masters of the universe](https://www.ft.com/content/e0bbbaf7-9f70-44cf-9bbf-077152d605a5)  - Financial Times![image](download_images/Image3.jpg)



---

4. [Metalcore issues sarcastic apology for using AI in a social media post, gets a subreddit ban](https://www.loudersound.com/news/ice-nine-kills-sarcastically-apologise-ai-image-2025)  - Louder![image](download_images/Image4.jpg)



---

5. 
Maybe your car could counsel you in a soothing voice when it detects road rage



---

6. [I could see studies fitting drivers with rage detection equipment and using AI to determine environmental causes of it](https://techxplore.com/news/2025-05-ai-sources-driver-stress-paving.html#google_vignette)  - TechXplore

---

7. ["concerning" refusal of cooperation from OpenAI models to simulated shutdown request](https://timesofindia.indiatimes.com/technology/artificial-intelligence/tesla-ceo-elon-musks-one-word-reply-to-openais-ai-model-refusing-to-shutdown-on-command/articleshow/121414780.cms)  - The Times of India![image](download_images/Image7.jpg)



---

8. [Could AI lead to hoax-driven panics? as opposed to chronic hoaxes like chemtrails and vaccines causing autism](https://www.statnews.com/2025/05/27/artificial-intelligence-bioterrorism-deepfake-public-health-threat/)  - STAT![image](download_images/Image8.jpg)



---

9. [FutureHouse's demonstrates 'AI scientist'](https://www.futurehouse.org/research-announcements/demonstrating-end-to-end-scientific-discovery-with-robin-a-multi-agent-system)  - www.futurehouse.org![image](download_images/Image9.jpg)



---

10. [Google is shifting from buying keywords, to AI placement](https://digiday.com/marketing/how-google-is-adapting-its-traditional-search-ads-for-ai-mode-and-ai-overviews-with-googles-dan-taylor/)  - Digiday![image](download_images/Image10.jpg)



---

11. [Self-Driving Truck Startup Siphoned Trade Secrets to Chinese Companies despite agreement with CFIUS](https://www.wsj.com/tech/china-self-driving-trucks-tusimple-c20255e1)  - The Wall Street Journal

---

12. [Working with robots often carries mental strain, studies find](https://www.ft.com/content/528e3c25-22c7-4c83-b80a-dd07dae92c5d)  - Financial Times![image](download_images/Image12.jpg)



---

13. [Researchers detail major GitHub MCP vulnerability](https://simonwillison.net/2025/May/26/github-mcp-exploited/)  - Simon Willison’s Weblog

---

14. [Waymo says sensors and lidar give its robotaxis superhuman abilities, especially in poor visibility](https://www.businessinsider.com/waymo-robotaxi-lidar-sensor-versus-cameras-vision-only-2025-5)  - Business Insider![image](download_images/Image14.jpg)



---

15. [Inside Amazon's sprawling warehouse robot factories](https://www.businessinsider.com/amazon-robotics-facilities-show-modern-us-manufacturing-2025-5?international=true&r=US&IR=T)  - Business Insider![image](download_images/Image15.jpg)



---

16. [Caninoids proliferate, from production line to front line](https://www.ft.com/content/14346b9e-2c04-4f56-9dc2-ab72e840d0db)  - Financial Times![image](download_images/Image16.jpg)



---

17. [Prediction: General-purpose AI could start getting worse](https://www.theregister.com/2025/05/27/opinion_column_ai_model_collapse/)  - The Register

---

18. [Nice FT robots supplement www.ft.com/roboticsguide](https://www.ft.com/roboticsguide)  - Financial Times![image](download_images/Image18.jpg)



---

19. [March of the cobots: the technology lowering the barrier to automation](https://www.ft.com/content/78c1d4e9-ad30-47f5-ab7b-390df5bc1f10)  - Financial Times![image](download_images/Image19.jpg)



---

20. [Humanoid robots spark debate about safety, form and function](https://www.ft.com/content/02f72125-dbc9-451d-84f8-1dc9e8bfb8ee)  - Financial Times![image](download_images/Image20.jpg)



---

21. [Ukraine’s ‘drone war’ hastens development of autonomous weapons](https://www.ft.com/content/165272fb-832f-4299-a0d2-1be8efcf5758)  - Financial Times![image](download_images/Image21.jpg)



---

22. [AGI confirmed](https://www.reddit.com/r/singularity/comments/1kw6c46/agi_confirmed/)  - Reddit

---

23. [AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox (gift link)](https://www.bloomberg.com/news/articles/2025-05-22/ai-tools-are-ceo-s-new-fix-for-the-post-vacation-inbox-mess)  - Bloomberg

---

24. [The GitHub MCP server can leak your private repos](https://x.com/lbeurerkellner/status/1926991491735429514)  - X

---

25. [Former Meta exec says asking artists for permission to use material doesn't scale and would "kill" the AI business](https://www.avclub.com/nick-clegg-ai-meta)  - The A.V. Club

---

26. [A Karen Hao interview](https://finance.yahoo.com/video/silicon-valleys-scale-costs-model-160052154.html)  - Yahoo Finance

---

27. [Semantic clinical AI using knowledge graphs passes a medical licensing exam](https://www.psychologytoday.com/us/blog/the-future-brain/202505/enhanced-llm-aces-the-us-medical-licensing-examination)  - Psychology Today

---

28. 
and we will atone



---

29. [You have meddled with the primal forces of nature.](https://www.youtube.com/watch?v=A64rR5Dp07s)  - YouTube![image](download_images/Image29.jpg)



---

30. [Tim Cook declined Middle East junket. Then Trump got back on his tariff phones and build them here BS. (although you wouldn't know from the limp headline)](https://www.nytimes.com/2025/05/26/technology/techs-trump-whisperer-tim-cook-goes-quiet-as-his-influence-fades.html)  - The New York Times

---

31. [The real cost of AI is being paid far from Silicon Valley: environmental degradation and human cost from mining, desertification from climate change](https://restofworld.org/2025/ai-resource-extraction-chile-indigenous-communities/)  - Rest of World![image](download_images/Image31.jpg)



---

32. [The Visitors: Humans react to alien visitors (Made with Veo 3)](https://www.reddit.com/r/ChatGPT/comments/1kvdv5k/the_visitors_humans_react_to_alien_visitors_made/)  - Reddit

---

33. [AI-powered shoe insoles revolutionize sports training and injury recovery](https://www.thebrighterside.news/post/ai-powered-shoe-insoles-revolutionize-sports-training-and-injury-recovery/)  - The Brighter Side of News![image](download_images/Image33.jpg)



---

34. [Eric Schmidt Says He Often Meets People Who Can't Tell Apart The Real World From The Online One, AI Will Only Make It Worse](https://www.benzinga.com/25/05/45604065/former-google-boss-eric-schmidt-says-ai-will-only-exacerbate-the-blurred-lines-between-online-perception-and-reality)  - Benzinga![image](download_images/Image34.jpg)



---

35. [Largest Ever UK Defence AI Trial Conducted Across Land, Sea](https://www.asdnews.com/news/defense/2025/05/26/largest-ever-uk-defence-ai-trial-conducted-across-land-sea-air)  - www.asdnews.com![image](download_images/Image35.jpg)



---

36. [Dwarkesh: How Does Claude 4 Think?](https://www.dwarkesh.com/p/sholto-trenton-2)  - www.dwarkesh.com![image](download_images/Image36.jpg)



---

37. [National Heart Centre Singapore unveils AI for rapid coronary artery disease prediction](https://www.healthcareitnews.com/news/asia/national-heart-centre-singapore-unveils-ai-rapid-coronary-artery-disease-prediction)  - Healthcare IT News![image](download_images/Image37.jpg)



---

38. [National Taiwan University Hospital fully deploys pancreatic cancer imaging AI](https://www.healthcareitnews.com/news/asia/national-taiwan-university-hospital-fully-deploys-pancreatic-cancer-imaging-ai)  - Healthcare IT News![image](download_images/Image38.jpg)



---

39. [Google's 'AI Mode' could be bad for Reddit, they pay Reddit but then summarize the relevant threads so you don't click through.](https://www.businessinsider.com/reddit-google-ai-mode-search-traffic-2025-5)  - Business Insider![image](download_images/Image39.jpg)



---

40. [Google Is Burying the Web Alive: AI, chatbots, and the end of the link economy.](https://nymag.com/intelligencer/article/google-ai-mode-search-results-bury-the-web.html)  - Intelligencer![image](download_images/Image40.jpg)



---

41. [Highlights from the Claude 4 system prompt - Simon Willison](https://simonwillison.net/2025/May/25/claude-4-system-prompt/)  - Simon Willison’s Weblog![image](download_images/Image41.jpg)



---

42. [Out of the 14 original creators of Meta's Llama AI model, 11 have left the company, with five joining French AI startup Mistral, founded by former Meta researchers including key Llama architects Guillaume Lample and Timothée Lacroix.](https://www.businessinsider.com/meta-llama-ai-talent-mistral-2025-5)  - Business Insider![image](download_images/Image42.jpg)



---

43. [Legora Boosts Funding by \\$80 Million For AI-Driven Legal Tech (that will hopefully not hallucinate citations and enrage judges)](https://news.bloomberglaw.com/legal-ops-and-tech/legora-boosts-funding-by-80-million-for-ai-driven-legal-tech)  - Bloomberg Law![image](download_images/Image43.jpg)



---

44. [White House releases TruthGPT, an AI that rewrites all social media posts globally to reflect undisputed facts. 'Fake news' problem solved!](https://www.instagram.com/reel/DKDiQHITdUh/)  - Instagram![image](download_images/Image44.jpg)



---

45. 
If humans came with a fully-developed lexer and parser, human language would be more like computer languages built on formal grammars, in fact LLMs seem to have built a more human-like model of language with all its idiosyncrasies.



---

46. [Chomsky is cranky about ChatGPT. LLMs show you don't *need* a universal grammar. True, humans learn from a million times less data, maybe this means some of the grammar capability is innately hard-wired. Maybe it means the learning algorithm is better and it builds the grammar stuff from scratch.](https://chomsky.info/20230503-2/?utm_source=chatgpt.com)  - chomsky.info

---

47. [always gotta go down at least 20 points first, just to make it interesting](https://www.theguardian.com/sport/2025/may/25/nba-playoffs-knicks-pacers-game-3)  - the Guardian![image](download_images/Image47.jpg)



---

48. 
Woke up this mor-ning 

Iykyk



---

49. [New newsletter format, any comments lmk](https://skynetandchill.beehiiv.com/p/ai-reading-for-sunday-may-25-00c7a6b1c57ba376)  - skynetandchill.beehiiv.com

---

50. [Musk asserts AI will make search redundant in comment on Google Search share dipping below 90%](https://www.tomshardware.com/tech-industry/artificial-intelligence/musk-asserts-ai-will-make-search-redundant-in-comment-on-google-search-share-dipping-below-90-percent)  - Tom's Hardware![image](download_images/Image50.jpg)



---

51. [New Device Mounts to Your Toilet to Analyze Your Turds Using the Power of AI. 

Gonna talk to you in AI TikTok voice and tell you to consume more fiber?](https://futurism.com/ai-throne-toilet)  - Futurism![image](download_images/Image51.jpg)



---

52. [Optimizing Assembly Code with LLMs: Reinforcement Learning Outperforms Traditional Compilers](https://www.marktechpost.com/2025/05/24/optimizing-assembly-code-with-llms-reinforcement-learning-outperforms-traditional-compilers/)  - MarkTechPost

---

53. [If you just 'count the server racks', will you know if DeepSeek is training on them? What's the counterfactual, if there was no Gulf deal? would they invest in US data centers? the techbros want planetary-scale compute to do 100 years of science in 10 years - is that real or a desert mirage?](https://www.chinatalk.media/p/about-that-ai-middle-east-deal)  - www.chinatalk.media![image](download_images/Image53.jpg)



---

54. [Google’s ‘world-model’ bet: building the AI operating layer before Microsoft captures the UI](https://venturebeat.com/ai/googles-world-model-bet-building-the-ai-operating-layer-before-microsoft-captures-the-ui/)  - VentureBeat![image](download_images/Image54.jpg)



---

55. [Google I/O showed off AI done right — hopefully, Apple was paying attention](https://www.tomsguide.com/ai/google-i-o-showed-off-ai-done-right-hopefully-apple-was-paying-attention)  - Tom's Guide![image](download_images/Image55.jpg)



---

56. [Chris Walti, Tesla's former lead for the Optimus robot, criticized humanoid robots like Optimus as unsuitable for most factory work, calling them a "ninth-inning robotics problem" while current technology is in the "third inning."](https://www.businessinsider.com/teslas-first-optimus-lead-doubts-about-elon-musks-robot-dream-2025-5)  - Business Insider![image](download_images/Image56.jpg)



---

57. [Can one person really build a billion-dollar business with AI agents? This author thinks so](https://www.zdnet.com/article/can-you-build-a-billion-dollar-business-with-only-ai-agents-yet-this-author-thinks-so/)  - ZDNET![image](download_images/Image57.jpg)



---

58. [Book excerpt: "Empire of AI" by Karen Hao](https://www.cbsnews.com/news/book-excerpt-empire-of-ai-by-karen-hao-artificial-intelligence-sam-altman-openai/)  - CBS News![image](download_images/Image58.jpg)



---

59. 




---

60. [Multi-agent systems are like containers and k8, powerful but makes systems harder to reason about, and anything you want to do you can probably do with a monolith. But some moonshots and architecture astronauts can probably go interesting places with them.](https://venturebeat.com/ai/beyond-single-model-ai-how-architectural-design-drives-reliable-multi-agent-orchestration/)  - VentureBeat![image](download_images/Image60.jpg)



---

61. [AI as a courtroom interpreter? Bill could make Wisconsin the first. {Maybe courtroom stenographers eventually.)](https://captimes.com/news/government/ai-as-a-courtroom-interpreter-bill-could-make-wisconsin-the-first/article_a947a10a-e8ab-4c70-b054-e251cf6bbcde.html)  - The Cap Times![image](download_images/Image61.jpg)



---

62. [AI Is Replacing Women's Jobs Specifically](https://futurism.com/ai-labor-gender-equity)  - Futurism![image](download_images/Image62.jpg)



---

63. [LinkedIn exec: AI automation of entry-level tasks is reducing junior roles traditionally used to climb the career ladder, hurting new grads](https://fortune.com/2025/05/25/ai-entry-level-jobs-gen-z-careers-young-workers-linkedin/)  - Fortune![image](download_images/Image63.jpg)



---

64. [Elon Musk and Tesla are going all in for the robotaxi race, despite some issues with running red lights and driving in the bike lane. (Linette Lopez, wherever you are, we miss you!)](https://www.businessinsider.com/tesla-waymo-robotaxi-austin-auto-car-bi-today-2025-5)  - Business Insider![image](download_images/Image64.jpg)



---

65. [Are robotaxis about to cross the chasm to mass adoption? Waymo, Tesla, Zoox expand offerings](https://www.digitaltrends.com/cars/robotaxi-news-readiness/)  - Digital Trends![image](download_images/Image65.jpg)



---

66. [Chrome AI Pro subscribers can now have Gemini view tabs, summarize pages, and interact via voice within the browser.](https://www.theverge.com/google/673659/gemini-google-chrome-integration-agentic-era)  - The Verge![image](download_images/Image66.jpg)



---

67. [‘Microsoft is the AI ringleader’: tech rivals flock to software giant’s stage](https://www.ft.com/content/9f6225c5-38e7-46a1-a920-e05a9a8df319)  - Financial Times![image](download_images/Image67.jpg)



---

68. [Doctors and nurses could wear smart glasses that warn them if they are about to give the wrong medicine or make other errors. See, a humanoid robot would never /s](https://www.nbcnews.com/health/health-news/medical-errors-are-still-harming-patients-ai-help-change-rcna205963)  - NBC News![image](download_images/Image68.jpg)



---

69. [A database of 117 legal cases so far where generative AI produced hallucinated citations submitted in court filings.](https://www.damiencharlotin.com/hallucinations/)  - www.damiencharlotin.com

---

70. [Simon Willison gives the Claude system card a read, including  a striking “spiritual bliss” attractor state... talking to itself, Claude gravitated to profuse gratitude and increasingly abstract and joyous spiritual or meditative expressions.](https://simonwillison.net/2025/May/25/claude-4-system-card/)  - Simon Willison’s Weblog![image](download_images/Image70.jpg)



---

71. [Quantized MoE models and other tricks are boosting efficiency](https://www.theregister.com/2025/05/25/ai_models_are_evolving/)  - The Register

---

72. [Xi Jinping’s plan to beat America at AI](https://www.economist.com/china/2025/05/25/xi-jinpings-plan-to-overtake-america-in-ai)  - The Economist![image](download_images/Image72.jpg)



---

73. 
Oh cool I got this schwag ordered before all the craziness, did not get hit up for a big tariff

I guess I should be grateful for incompetence



---

74. 
Double rainbow!



---

75. [At Amazon, Some Coders Say Their Jobs Have Begun to Resemble Warehouse Work](https://www.nytimes.com/2025/05/25/business/amazon-ai-coders.html)  - The New York Times

---

76. [Use AI To Disrupt, Not Sell To Enterprises, Venture Capitalists Urge](https://www.forbes.com/sites/joemckendrick/2025/05/25/use-ai-to-disrupt-not-sell-to-enterprises-venture-capitalists-urge/)  - Forbes![image](download_images/Image76.jpg)



---

77. [Nvidia to launch cheaper Blackwell AI chip for China after US export curbs, sources say](https://www.reuters.com/world/china/nvidia-launch-cheaper-blackwell-ai-chip-china-after-us-export-curbs-sources-say-2025-05-24/)  - Reuters

---

78. [NYC crypto bro charged with kidnapping and torturing Italian for weeks to get crypto passwords](https://www.nytimes.com/2025/05/24/nyregion/crypto-investor-torture-italian-tourist.html)  - The New York Times

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [15]:
stop_index = 32

In [16]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))

0. [Using AI to community-note crypto scams on social media](https://indicator.media/p/the-top-community-notes-contributor-is-an-antivirus-startup-9354)  - indicator.media

---

1. [Apple said to plan smart home device later this year despite Apple Intelligence delays](https://www.bloomberg.com/news/newsletters/2025-05-25/how-jony-ive-openai-deal-will-impact-apple-new-details-on-apple-s-ios-redesign-mb3lwu45?cmpid=BBD052525_POWERON&utm_medium=email&utm_source=newsletter&utm_term=250525&utm_campaign=poweron)  - Bloomberg

---

2. 
is nothing sacred, not even shilling your metalcore merch and swag drop



---

3. [Creator of Succession has a new œuvre about tech masters of the universe](https://www.ft.com/content/e0bbbaf7-9f70-44cf-9bbf-077152d605a5)  - Financial Times![image](download_images/Image3.jpg)



---

4. [Metalcore issues sarcastic apology for using AI in a social media post, gets a subreddit ban](https://www.loudersound.com/news/ice-nine-kills-sarcastically-apologise-ai-image-2025)  - Louder![image](download_images/Image4.jpg)



---

5. 
Maybe your car could counsel you in a soothing voice when it detects road rage



---

6. [I could see studies fitting drivers with rage detection equipment and using AI to determine environmental causes of it](https://techxplore.com/news/2025-05-ai-sources-driver-stress-paving.html#google_vignette)  - TechXplore

---

7. ["concerning" refusal of cooperation from OpenAI models to simulated shutdown request](https://timesofindia.indiatimes.com/technology/artificial-intelligence/tesla-ceo-elon-musks-one-word-reply-to-openais-ai-model-refusing-to-shutdown-on-command/articleshow/121414780.cms)  - The Times of India![image](download_images/Image7.jpg)



---

8. [Could AI lead to hoax-driven panics? as opposed to chronic hoaxes like chemtrails and vaccines causing autism](https://www.statnews.com/2025/05/27/artificial-intelligence-bioterrorism-deepfake-public-health-threat/)  - STAT![image](download_images/Image8.jpg)



---

9. [FutureHouse's demonstrates 'AI scientist'](https://www.futurehouse.org/research-announcements/demonstrating-end-to-end-scientific-discovery-with-robin-a-multi-agent-system)  - www.futurehouse.org![image](download_images/Image9.jpg)



---

10. [Google is shifting from buying keywords, to AI placement](https://digiday.com/marketing/how-google-is-adapting-its-traditional-search-ads-for-ai-mode-and-ai-overviews-with-googles-dan-taylor/)  - Digiday![image](download_images/Image10.jpg)



---

11. [Self-Driving Truck Startup Siphoned Trade Secrets to Chinese Companies despite agreement with CFIUS](https://www.wsj.com/tech/china-self-driving-trucks-tusimple-c20255e1)  - The Wall Street Journal

---

12. [Working with robots often carries mental strain, studies find](https://www.ft.com/content/528e3c25-22c7-4c83-b80a-dd07dae92c5d)  - Financial Times![image](download_images/Image12.jpg)



---

13. [Researchers detail major GitHub MCP vulnerability](https://simonwillison.net/2025/May/26/github-mcp-exploited/)  - Simon Willison’s Weblog

---

14. [Waymo says sensors and lidar give its robotaxis superhuman abilities, especially in poor visibility](https://www.businessinsider.com/waymo-robotaxi-lidar-sensor-versus-cameras-vision-only-2025-5)  - Business Insider![image](download_images/Image14.jpg)



---

15. [Inside Amazon's sprawling warehouse robot factories](https://www.businessinsider.com/amazon-robotics-facilities-show-modern-us-manufacturing-2025-5?international=true&r=US&IR=T)  - Business Insider![image](download_images/Image15.jpg)



---

16. [Caninoids proliferate, from production line to front line](https://www.ft.com/content/14346b9e-2c04-4f56-9dc2-ab72e840d0db)  - Financial Times![image](download_images/Image16.jpg)



---

17. [Prediction: General-purpose AI could start getting worse](https://www.theregister.com/2025/05/27/opinion_column_ai_model_collapse/)  - The Register

---

18. [Nice FT robots supplement www.ft.com/roboticsguide](https://www.ft.com/roboticsguide)  - Financial Times![image](download_images/Image18.jpg)



---

19. [March of the cobots: the technology lowering the barrier to automation](https://www.ft.com/content/78c1d4e9-ad30-47f5-ab7b-390df5bc1f10)  - Financial Times![image](download_images/Image19.jpg)



---

20. [Humanoid robots spark debate about safety, form and function](https://www.ft.com/content/02f72125-dbc9-451d-84f8-1dc9e8bfb8ee)  - Financial Times![image](download_images/Image20.jpg)



---

21. [Ukraine’s ‘drone war’ hastens development of autonomous weapons](https://www.ft.com/content/165272fb-832f-4299-a0d2-1be8efcf5758)  - Financial Times![image](download_images/Image21.jpg)



---

22. [AGI confirmed](https://www.reddit.com/r/singularity/comments/1kw6c46/agi_confirmed/)  - Reddit

---

23. [AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox (gift link)](https://www.bloomberg.com/news/articles/2025-05-22/ai-tools-are-ceo-s-new-fix-for-the-post-vacation-inbox-mess)  - Bloomberg

---

24. [The GitHub MCP server can leak your private repos](https://x.com/lbeurerkellner/status/1926991491735429514)  - X

---

25. [Former Meta exec says asking artists for permission to use material doesn't scale and would "kill" the AI business](https://www.avclub.com/nick-clegg-ai-meta)  - The A.V. Club

---

26. [A Karen Hao interview](https://finance.yahoo.com/video/silicon-valleys-scale-costs-model-160052154.html)  - Yahoo Finance

---

27. [Semantic clinical AI using knowledge graphs passes a medical licensing exam](https://www.psychologytoday.com/us/blog/the-future-brain/202505/enhanced-llm-aces-the-us-medical-licensing-examination)  - Psychology Today

---

28. 
and we will atone



---

29. [You have meddled with the primal forces of nature.](https://www.youtube.com/watch?v=A64rR5Dp07s)  - YouTube![image](download_images/Image29.jpg)



---

30. [Tim Cook declined Middle East junket. Then Trump got back on his tariff phones and build them here BS. (although you wouldn't know from the limp headline)](https://www.nytimes.com/2025/05/26/technology/techs-trump-whisperer-tim-cook-goes-quiet-as-his-influence-fades.html)  - The New York Times

---

31. [The real cost of AI is being paid far from Silicon Valley: environmental degradation and human cost from mining, desertification from climate change](https://restofworld.org/2025/ai-resource-extraction-chile-indigenous-communities/)  - Rest of World![image](download_images/Image31.jpg)



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [17]:
import markdown
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)

[Using AI to community-note crypto scams on social media](https://indicator.media/p/the-top-community-notes-contributor-is-an-antivirus-startup-9354)  - indicator.media

---

[Apple said to plan smart home device later this year despite Apple Intelligence delays](https://www.bloomberg.com/news/newsletters/2025-05-25/how-jony-ive-openai-deal-will-impact-apple-new-details-on-apple-s-ios-redesign-mb3lwu45?cmpid=BBD052525_POWERON&utm_medium=email&utm_source=newsletter&utm_term=250525&utm_campaign=poweron)  - Bloomberg

---


is nothing sacred, not even shilling your metalcore merch and swag drop



---



![image](download_images/Image3.jpg)

[Creator of Succession has a new œuvre about tech masters of the universe](https://www.ft.com/content/e0bbbaf7-9f70-44cf-9bbf-077152d605a5)  - Financial Times

---



![image](download_images/Image4.jpg)

[Metalcore issues sarcastic apology for using AI in a social media post, gets a subreddit ban](https://www.loudersound.com/news/ice-nine-kills-sarcastically-apologise-ai-image-2025)  - Louder

---


Maybe your car could counsel you in a soothing voice when it detects road rage



---

[I could see studies fitting drivers with rage detection equipment and using AI to determine environmental causes of it](https://techxplore.com/news/2025-05-ai-sources-driver-stress-paving.html#google_vignette)  - TechXplore

---



![image](download_images/Image7.jpg)

["concerning" refusal of cooperation from OpenAI models to simulated shutdown request](https://timesofindia.indiatimes.com/technology/artificial-intelligence/tesla-ceo-elon-musks-one-word-reply-to-openais-ai-model-refusing-to-shutdown-on-command/articleshow/121414780.cms)  - The Times of India

---



![image](download_images/Image8.jpg)

[Could AI lead to hoax-driven panics? as opposed to chronic hoaxes like chemtrails and vaccines causing autism](https://www.statnews.com/2025/05/27/artificial-intelligence-bioterrorism-deepfake-public-health-threat/)  - STAT

---



![image](download_images/Image9.jpg)

[FutureHouse's demonstrates 'AI scientist'](https://www.futurehouse.org/research-announcements/demonstrating-end-to-end-scientific-discovery-with-robin-a-multi-agent-system)  - www.futurehouse.org

---



![image](download_images/Image10.jpg)

[Google is shifting from buying keywords, to AI placement](https://digiday.com/marketing/how-google-is-adapting-its-traditional-search-ads-for-ai-mode-and-ai-overviews-with-googles-dan-taylor/)  - Digiday

---

[Self-Driving Truck Startup Siphoned Trade Secrets to Chinese Companies despite agreement with CFIUS](https://www.wsj.com/tech/china-self-driving-trucks-tusimple-c20255e1)  - The Wall Street Journal

---



![image](download_images/Image12.jpg)

[Working with robots often carries mental strain, studies find](https://www.ft.com/content/528e3c25-22c7-4c83-b80a-dd07dae92c5d)  - Financial Times

---

[Researchers detail major GitHub MCP vulnerability](https://simonwillison.net/2025/May/26/github-mcp-exploited/)  - Simon Willison’s Weblog

---



![image](download_images/Image14.jpg)

[Waymo says sensors and lidar give its robotaxis superhuman abilities, especially in poor visibility](https://www.businessinsider.com/waymo-robotaxi-lidar-sensor-versus-cameras-vision-only-2025-5)  - Business Insider

---



![image](download_images/Image15.jpg)

[Inside Amazon's sprawling warehouse robot factories](https://www.businessinsider.com/amazon-robotics-facilities-show-modern-us-manufacturing-2025-5?international=true&r=US&IR=T)  - Business Insider

---



![image](download_images/Image16.jpg)

[Caninoids proliferate, from production line to front line](https://www.ft.com/content/14346b9e-2c04-4f56-9dc2-ab72e840d0db)  - Financial Times

---

[Prediction: General-purpose AI could start getting worse](https://www.theregister.com/2025/05/27/opinion_column_ai_model_collapse/)  - The Register

---



![image](download_images/Image18.jpg)

[Nice FT robots supplement www.ft.com/roboticsguide](https://www.ft.com/roboticsguide)  - Financial Times

---



![image](download_images/Image19.jpg)

[March of the cobots: the technology lowering the barrier to automation](https://www.ft.com/content/78c1d4e9-ad30-47f5-ab7b-390df5bc1f10)  - Financial Times

---



![image](download_images/Image20.jpg)

[Humanoid robots spark debate about safety, form and function](https://www.ft.com/content/02f72125-dbc9-451d-84f8-1dc9e8bfb8ee)  - Financial Times

---



![image](download_images/Image21.jpg)

[Ukraine’s ‘drone war’ hastens development of autonomous weapons](https://www.ft.com/content/165272fb-832f-4299-a0d2-1be8efcf5758)  - Financial Times

---

[AGI confirmed](https://www.reddit.com/r/singularity/comments/1kw6c46/agi_confirmed/)  - Reddit

---

[AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox (gift link)](https://www.bloomberg.com/news/articles/2025-05-22/ai-tools-are-ceo-s-new-fix-for-the-post-vacation-inbox-mess)  - Bloomberg

---

[The GitHub MCP server can leak your private repos](https://x.com/lbeurerkellner/status/1926991491735429514)  - X

---

[Former Meta exec says asking artists for permission to use material doesn't scale and would "kill" the AI business](https://www.avclub.com/nick-clegg-ai-meta)  - The A.V. Club

---

[A Karen Hao interview](https://finance.yahoo.com/video/silicon-valleys-scale-costs-model-160052154.html)  - Yahoo Finance

---

[Semantic clinical AI using knowledge graphs passes a medical licensing exam](https://www.psychologytoday.com/us/blog/the-future-brain/202505/enhanced-llm-aces-the-us-medical-licensing-examination)  - Psychology Today

---


and we will atone



---



![image](download_images/Image29.jpg)

[You have meddled with the primal forces of nature.](https://www.youtube.com/watch?v=A64rR5Dp07s)  - YouTube

---

[Tim Cook declined Middle East junket. Then Trump got back on his tariff phones and build them here BS. (although you wouldn't know from the limp headline)](https://www.nytimes.com/2025/05/26/technology/techs-trump-whisperer-tim-cook-goes-quiet-as-his-influence-fades.html)  - The New York Times

---



![image](download_images/Image31.jpg)

[The real cost of AI is being paid far from Silicon Valley: environmental degradation and human cost from mining, desertification from climate change](https://restofworld.org/2025/ai-resource-extraction-chile-indigenous-communities/)  - Rest of World

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

7415

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import RootModel
import os

from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")


system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most imporant story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

user_prompt = """
{input_text}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

model = ChatOpenAI(
        model="gpt-4.1",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
# Create the chain
chain = prompt_template | model.with_structured_output(ItemList)
response = chain.invoke({"input_text": output_df.to_json(orient="records")})


In [19]:
len(response.items)


32

In [20]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[Researchers detail major GitHub MCP vulnerability](https://simonwillison.net/2025/May/26/github-mcp-exploited/)  - Simon Willison’s Weblog

---

[The GitHub MCP server can leak your private repos](https://x.com/lbeurerkellner/status/1926991491735429514)  - X

---



![image](download_images/Image10.jpg)

[Google is shifting from buying keywords, to AI placement](https://digiday.com/marketing/how-google-is-adapting-its-traditional-search-ads-for-ai-mode-and-ai-overviews-with-googles-dan-taylor/)  - Digiday

---

[Self-Driving Truck Startup Siphoned Trade Secrets to Chinese Companies despite agreement with CFIUS](https://www.wsj.com/tech/china-self-driving-trucks-tusimple-c20255e1)  - The Wall Street Journal

---



![image](download_images/Image31.jpg)

[The real cost of AI is being paid far from Silicon Valley: environmental degradation and human cost from mining, desertification from climate change](https://restofworld.org/2025/ai-resource-extraction-chile-indigenous-communities/)  - Rest of World

---

[Semantic clinical AI using knowledge graphs passes a medical licensing exam](https://www.psychologytoday.com/us/blog/the-future-brain/202505/enhanced-llm-aces-the-us-medical-licensing-examination)  - Psychology Today

---



![image](download_images/Image7.jpg)

["concerning" refusal of cooperation from OpenAI models to simulated shutdown request](https://timesofindia.indiatimes.com/technology/artificial-intelligence/tesla-ceo-elon-musks-one-word-reply-to-openais-ai-model-refusing-to-shutdown-on-command/articleshow/121414780.cms)  - The Times of India

---



![image](download_images/Image8.jpg)

[Could AI lead to hoax-driven panics? as opposed to chronic hoaxes like chemtrails and vaccines causing autism](https://www.statnews.com/2025/05/27/artificial-intelligence-bioterrorism-deepfake-public-health-threat/)  - STAT

---



![image](download_images/Image9.jpg)

[FutureHouse's demonstrates 'AI scientist'](https://www.futurehouse.org/research-announcements/demonstrating-end-to-end-scientific-discovery-with-robin-a-multi-agent-system)  - www.futurehouse.org

---



![image](download_images/Image21.jpg)

[Ukraine’s ‘drone war’ hastens development of autonomous weapons](https://www.ft.com/content/165272fb-832f-4299-a0d2-1be8efcf5758)  - Financial Times

---



![image](download_images/Image16.jpg)

[Caninoids proliferate, from production line to front line](https://www.ft.com/content/14346b9e-2c04-4f56-9dc2-ab72e840d0db)  - Financial Times

---



![image](download_images/Image20.jpg)

[Humanoid robots spark debate about safety, form and function](https://www.ft.com/content/02f72125-dbc9-451d-84f8-1dc9e8bfb8ee)  - Financial Times

---



![image](download_images/Image19.jpg)

[March of the cobots: the technology lowering the barrier to automation](https://www.ft.com/content/78c1d4e9-ad30-47f5-ab7b-390df5bc1f10)  - Financial Times

---



![image](download_images/Image18.jpg)

[Nice FT robots supplement www.ft.com/roboticsguide](https://www.ft.com/roboticsguide)  - Financial Times

---



![image](download_images/Image12.jpg)

[Working with robots often carries mental strain, studies find](https://www.ft.com/content/528e3c25-22c7-4c83-b80a-dd07dae92c5d)  - Financial Times

---



![image](download_images/Image15.jpg)

[Inside Amazon's sprawling warehouse robot factories](https://www.businessinsider.com/amazon-robotics-facilities-show-modern-us-manufacturing-2025-5?international=true&r=US&IR=T)  - Business Insider

---

[Tim Cook declined Middle East junket. Then Trump got back on his tariff phones and build them here BS. (although you wouldn't know from the limp headline)](https://www.nytimes.com/2025/05/26/technology/techs-trump-whisperer-tim-cook-goes-quiet-as-his-influence-fades.html)  - The New York Times

---

[Apple said to plan smart home device later this year despite Apple Intelligence delays](https://www.bloomberg.com/news/newsletters/2025-05-25/how-jony-ive-openai-deal-will-impact-apple-new-details-on-apple-s-ios-redesign-mb3lwu45?cmpid=BBD052525_POWERON&utm_medium=email&utm_source=newsletter&utm_term=250525&utm_campaign=poweron)  - Bloomberg

---

[AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox (gift link)](https://www.bloomberg.com/news/articles/2025-05-22/ai-tools-are-ceo-s-new-fix-for-the-post-vacation-inbox-mess)  - Bloomberg

---

[Former Meta exec says asking artists for permission to use material doesn't scale and would "kill" the AI business](https://www.avclub.com/nick-clegg-ai-meta)  - The A.V. Club

---

[A Karen Hao interview](https://finance.yahoo.com/video/silicon-valleys-scale-costs-model-160052154.html)  - Yahoo Finance

---



![image](download_images/Image14.jpg)

[Waymo says sensors and lidar give its robotaxis superhuman abilities, especially in poor visibility](https://www.businessinsider.com/waymo-robotaxi-lidar-sensor-versus-cameras-vision-only-2025-5)  - Business Insider

---

[Using AI to community-note crypto scams on social media](https://indicator.media/p/the-top-community-notes-contributor-is-an-antivirus-startup-9354)  - indicator.media

---

[I could see studies fitting drivers with rage detection equipment and using AI to determine environmental causes of it](https://techxplore.com/news/2025-05-ai-sources-driver-stress-paving.html#google_vignette)  - TechXplore

---


Maybe your car could counsel you in a soothing voice when it detects road rage



---



![image](download_images/Image4.jpg)

[Metalcore issues sarcastic apology for using AI in a social media post, gets a subreddit ban](https://www.loudersound.com/news/ice-nine-kills-sarcastically-apologise-ai-image-2025)  - Louder

---


is nothing sacred, not even shilling your metalcore merch and swag drop



---


and we will atone



---

[AGI confirmed](https://www.reddit.com/r/singularity/comments/1kw6c46/agi_confirmed/)  - Reddit

---



![image](download_images/Image3.jpg)

[Creator of Succession has a new œuvre about tech masters of the universe](https://www.ft.com/content/e0bbbaf7-9f70-44cf-9bbf-077152d605a5)  - Financial Times

---



![image](download_images/Image29.jpg)

[You have meddled with the primal forces of nature.](https://www.youtube.com/watch?v=A64rR5Dp07s)  - YouTube

---

[Prediction: General-purpose AI could start getting worse](https://www.theregister.com/2025/05/27/opinion_column_ai_model_collapse/)  - The Register

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

7415

In [21]:
pyperclip.copy( output_str )


In [ ]:
date_str = data.feed[0].post.record.created_at
dt = datetime.fromisoformat(date_str.rstrip('Z'))
dt

In [ ]:

# Get the current datetime
now = datetime.now()

# Create a timedelta of 30 hours
delta = timedelta(hours=30)

# Subtract 30 hours from the current datetime
thirty_hours_ago = now - delta

print(thirty_hours_ago)


In [ ]:
from typing import TypedDict, Annotated

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, }  # 


In [ ]:
a

In [ ]:
import numpy as np

def decay_exponential(t, N0):
    k = np.log(2)  
    return N0 * np.exp(-k * t)

2*decay_exponential(4,1)-1

In [ ]:
```mermaid
flowchart LR

A[Hard] -->|Text| B(Round)
B --> C{Decision}
C -->|One| D[Result 1]
C -->|Two| E[Result 2]


In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

dt_str = '2025-05-24 16:00'
naive_dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M')

# CORRECT: Attach US Eastern timezone *with DST handled*
eastern_dt = naive_dt.replace(tzinfo=ZoneInfo("America/New_York"))

# Convert to UTC
utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
print(utc_dt)

In [ ]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions

import chromadb

# Set up the client
client = chromadb.PersistentClient(
    path="chromadb/chroma_articles/", settings=Settings(allow_reset=True)
)


# Get the collection
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY")
)

collection = client.get_collection("articles", embedding_function=openai_ef)

# Count the documents
num_docs = collection.count()
print(f"Number of documents in 'articles': {num_docs}")

In [ ]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

before_date = '2025-05-25 16:30'
if before_date:
    naive_dt = datetime.strptime(before_date, '%Y-%m-%d %H:%M')
    eastern_dt = naive_dt.replace(
        tzinfo=ZoneInfo("America/New_York"))
    utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
    before_date = utc_dt.strftime('%Y-%m-%d %H:%M')


In [ ]:
doc = collection.get(limit=1)
doc_str = doc['documents'][0]
doc_str2 = "\n".join(doc_str.split("\n")[12:])

In [ ]:
results = collection.query(
    query_texts=[doc_str2],
    n_results=1,  # number of similar documents to return
    include=["distances", "metadatas"],
#     where={'created': {'$lt': 2025-05-25 20:30'}
)

In [ ]:
results


In [ ]:
before_date


In [ ]:
                where_filter = {"created": f"$lt: {before_date}"}


In [ ]:
where_filter

In [ ]:
from typing import List 
import numpy as np 

def mmr(
    ratings: List[float],
    doc_embeddings: List[np.ndarray],
    lambda_param: float = 0.5,
    top_k: int = 50
) -> List[int]:
    """MMR (Maximal Marginal Relevance) algorithm for selecting a diverse set of documents.
    Combines normalized score and novelty vs previous items

    Args:
        doc_embeddings (List[np.ndarray]): embeddings of documents
        ratings (List[float]): ratings of documents, should be positive, we will scale to max 1
        lambda_param (float, optional): balance between rating and novelty. Defaults to 0.5.
        top_k (int, optional): number of documents to select. Defaults to 50.

    Returns:
        List[int]: indices of selected documents
    """
    selected = []
    candidates = list(range(len(doc_embeddings)))

    ratings = [rating / max(ratings) for rating in ratings]

    while len(selected) < top_k and candidates:
        if not selected:  # none yet
            # start with highest scoring document
            idx = np.argmax(ratings)
            selected.append(idx)  # add to selected
            candidates.remove(idx)  # remove from candidates
            continue

        scores = []
        for idx in candidates:
            rating = ratings[idx]
            # find most similar selected document to this doc idx
            # first array has 1 element so we get a 1x array
            max_sim_to_selected = max(cosine_similarity(
                [doc_embeddings[idx]],  # array with one doc
                [doc_embeddings[i]
                    for i in selected]  # array with all selected docs
            )[0])
            mmr_score = lambda_param * rating - \
                (1 - lambda_param) * max_sim_to_selected
            scores.append((mmr_score, idx))

        # Select candidate with highest MMR score
        # scores is a list of (score, idx), max does lexicographic sort so we get the highest score
        _, best_idx = max(scores)
        selected.append(best_idx)
        candidates.remove(best_idx)

    return selected